In [14]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell'
import pyspark;
sc = pyspark.SparkContext.getOrCreate();
from pyspark.sql import SparkSession;
spark = SparkSession(sc)

In [15]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType
eventSchema = (StructType()
  .add('InvoiceNo', StringType())
  .add('StockCode', StringType())
  .add('Description', StringType())
  .add('Quantity', IntegerType())
  .add('InvoiceDate', StringType())
  .add('UnitPrice', DoubleType())
  .add('CustomerID', IntegerType())
  .add('Country', StringType())
)

In [16]:
kafka_df = (spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "retail-events")
    .option("startingOffsets", "earliest")
    .load()
)

In [22]:
query = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
    .writeStream \
    .format("console") \
    .option("checkpointLocation", "path/to/HDFS/dir") \
    .start()

22/05/21 20:53:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-------+
| key|  value|
+----+-------+
|null|testing|
|null|   test|
|null|   test|
|null|   hiya|
|null|   more|
|null|   test|
|null|   test|
|null|   test|
+----+-------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
| key|value|
+----+-----+
|null| tttt|
+----+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+---------+
| key|    value|
+----+---------+
|null|yyyyyyyyy|
+----+---------+



In [24]:
query.stop()

In [25]:
query.stop()

In [26]:
from pyspark.sql.functions import col, from_json, to_date
retail_df = (kafka_df\
   .select(from_json(col("value").cast(StringType()), eventSchema).alias("message"), col("timestamp").alias("EventTime"))\
   .select("message.*", "EventTime")
)

In [27]:
base_path = "/tmp/data-lake/retail_events.parquet"
(retail_df
    .withColumn("EventDate", to_date(retail_df.EventTime))
    .writeStream
    .format('parquet')
    .outputMode("append")
    .trigger(once=True)
    .option('checkpointLocation', base_path + '/_checkpoint')
    .start(base_path)
)

22/05/21 21:12:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [28]:
df2 = spark.read.parquet("/tmp/data-lake/retail_events.parquet")